In [1]:
import os
import time
import enoki as ek

import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import xml, Thread, Transform4f, Bitmap, Float, Vector3f, UInt32
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap, Adam, SGD

2021-06-09 12:25:27 INFO main [optix_api.cpp:56] Dynamic loading of the Optix library ..

In [2]:
# Convert flat array into a vector of arrays (will be included in next enoki release)
def ravel(buf, dim = 3):
    idx = dim * UInt32.arange(ek.slices(buf) // dim)
    return Vector3f(ek.gather(buf, idx), ek.gather(buf, idx + 1), ek.gather(buf, idx + 2))

# Return contiguous flattened array (will be included in next enoki release)
def unravel(source, target, dim = 3):
    idx = UInt32.arange(ek.slices(source))
    for i in range(dim):
        ek.scatter(target, source[i], dim * idx + i)

In [3]:
# Prepare output folder
output_path = "output/invert_pose/"
if not os.path.isdir(output_path):
    os.makedirs(output_path)

# Load example scene
scene_folder = '../resources/data/docs/examples/invert_pose/'
Thread.thread().file_resolver().append(scene_folder)
scene = xml.load_file(scene_folder + 'scene.xml')

params = traverse(scene)
print(params)
positions_buf = params['object.vertex_positions_buf']
positions_initial = ravel(positions_buf)

# Create differential parameter to be optimized
scale_ref = Vector3f(1.0, 1.0, 2.0)

# Create a new ParameterMap (or dict)
params_optim = {
    "scale" : scale_ref,
}

# Construct an Adam optimizer that will adjust the translation parameters
opt = Adam(params_optim, lr=0.02)

# Apply the transformation to mesh vertex position and update scene (e.g. Optix BVH)
def apply_transformation():
    trasfo = Transform4f.scale(params_optim["scale"])
    new_positions = trasfo.transform_point(positions_initial)
    unravel(new_positions, params['object.vertex_positions_buf'])
    params.set_dirty('object.vertex_positions_buf')
    params.update()

# Render a reference image (no derivatives used yet)
apply_transformation()
image_ref = render(scene, spp=16)
crop_size = scene.sensors()[0].film().crop_size()

write_bitmap(output_path + 'out_ref.png', image_ref, crop_size)
print("Write " + output_path + "out_ref.png")

2021-06-09 12:25:27 INFO main [xml.cpp:1182] Loading XML file "../resources/data/docs/examples/invert_pose/scene.xml" ..

2021-06-09 12:25:27 INFO main [xml.cpp:1183] Using variant "gpu_autodiff_rgb"

2021-06-09 12:25:27 INFO main [PluginManager] Loading plugin "plugins/uniform.so" ..

2021-06-09 12:25:27 INFO main [PluginManager] Loading plugin "plugins/srgb.so" ..

2021-06-09 12:25:27 INFO main [PluginManager] Loading plugin "plugins/pathreparam.so" ..

...

2021-06-09 12:25:27 INFO main [PluginManager] Loading plugin "plugins/checkerboard.so" ..

2021-06-09 12:25:27 DEBUG main [OBJMesh] Loading mesh from "smooth_ramp.obj" ..

2021-06-09 12:25:27 DEBUG main [OBJMesh] "smooth_ramp.obj": read 1800 faces, 961 vertices (51.1 KiB in 3ms)

2021-06-09 12:25:27 INFO main [Scene] Building scene in OptiX ..

ParameterMap[
    PerspectiveCamera.near_clip,
    PerspectiveCamera.far_clip,
    PerspectiveCamera.focus_distance,
    PerspectiveCamera.shutter_open,
    PerspectiveCamera.shutter_open_time,
    Rectangle.to_world,
  * Rectangle.bsdf.reflectance.value,
  * Rectangle.emitter.radiance.value,
    object.to_world,
  * object.bsdf.reflectance.value,
    object.vertex_count,
    object.face_count,
  * object.faces_buf,
  * object.vertex_positions_buf,
  * object.vertex_normals_buf,
  * object.vertex_texcoords_buf,
    planemesh.to_world,
    planemesh.bsdf.reflectance.transform,
  * planemesh.bsdf.reflectance.color0.value,
  * planemesh.bsdf.reflectance.color1.value,
    planemesh.vertex_count,
    planemesh.face_count,
  * planemesh.faces_buf,
  * planemesh.vertex_positions_buf,
  * planemesh.vertex_normals_buf,
  * planemesh.vertex_texcoords_buf,
]
Write output/invert_pose/out_ref.png


In [4]:
print(scene)

Scene[
  children = [
    PathReparamIntegrator[
      max_depth = 3,
      rr_depth = 5
    ],
    PerspectiveCamera[
      x_fov = 35,
      near_clip = 0.01,
      far_clip = 10000,
      focus_distance = 10000,
      film = HDRFilm[
        size = [128, 128],
        crop_size = [128, 128],
        crop_offset = [0, 0],
        high_quality_edges = 0,
        filter = GaussianFilter[stddev=0.50, radius=2.00],
        file_format = OpenEXR,
        pixel_format = rgba,
        component_format = float16,
        dest_file = ""
      ],
      sampler = IndependentSampler[
        sample_count = 8
      ],
      resolution = [128, 128],
      shutter_open = 0,
      shutter_open_time = 0,
      aspect = 1,
      world_transform = AnimatedTransform[
        m_transform = [[0, -0.371391, -0.928477, 5],
                       [0, 0.928477, -0.371391, 2],
                       [1, 0, 0, 0],
                       [0, 0, 0, 1]],
        m_keyframes = []
      ]
    ],
    Rectangle[
     

In [4]:
# Move object before starting the optimization process
params_optim["scale"] = Vector3f(2.0, 2.0, 1.0)

time_a = time.time()

iterations = 1000
for it in range(iterations):
    # Perform a differentiable rendering of the scene
    image = render(scene,
                   optimizer=opt,
                   spp=4,
                   unbiased=True,
                   pre_render_callback=apply_transformation)

    write_bitmap(output_path + 'out_%03i.png' % it, image, crop_size)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step -> update displacement map
    opt.step()

    print('Iteration %03i: error=%g' % (it, ob_val[0]), end='\r')

time_b = time.time()

print()
print('%f ms per iteration' % (((time_b - time_a) * 1000) / iterations))

KeyboardInterrupt: 